In [1]:
import pandas as pd
import numpy as np
from pybedtools import BedTool
import matplotlib.pyplot as plt

In [5]:
peak_file = "/clusterfs/jgi/groups/gentech/homes/romalley/full_DAPseq_annotation/test_peak.bed"  # Input peak file with 'tf' column
gene_file = "/clusterfs/jgi/groups/gentech/seqtech/plant_multidap_data/genomes/annotations/Arabidopsis_thaliana_Col-0_cds_primary.gff"  # File with gene coordinates

# Load the gene coordinates and peaks as Pandas DataFrames
genes = pd.read_csv(gene_file, sep="\t", header=None, names=["chrom", "source", "region", "start", "end", "score", "strand", "idx", "gene_name"])
peaks = pd.read_csv(peak_file, sep="\t", header=0)


In [7]:
# Function to generate common bins (as defined earlier)
def generate_bins(start, end, num_bins, bin_prefix):
    bin_size = (end - start) // num_bins
    return [(start + i * bin_size, start + (i + 1) * bin_size, f"{bin_prefix}_{i+1}") for i in range(num_bins)]

# List to store all bins
all_bins = []

# Loop through each gene and align to common bins
for _, row in genes.iterrows():
    chrom = row["chrom"]
    gene_start, gene_end = row["start"], row["end"]
    strand = row["strand"]

    # Define regions for upstream, genebody, and downstream
    if strand == "+":
        upstream_start, upstream_end = gene_start - 2000, gene_start
        downstream_start, downstream_end = gene_end, gene_end + 2000
    else:  # Reverse for "-" strand
        upstream_start, upstream_end = gene_end, gene_end + 2000
        downstream_start, downstream_end = gene_start - 2000, gene_start
        gene_start, gene_end = gene_end, gene_start  # Flip start/end for consistency

    # Generate bins for upstream, genebody, and downstream
    upstream_bins = generate_bins(upstream_start, upstream_end, 20, "bin")
    genebody_bins = generate_bins(gene_start, gene_end, 10, "bin")
    downstream_bins = generate_bins(downstream_start, downstream_end, 20, "bin")

    # Add chromosome and bin coordinates to the list
    for start, end, bin_name in upstream_bins + genebody_bins + downstream_bins:
        all_bins.append((chrom, start, end, bin_name))

# Create a DataFrame of all common bins
bins_df = pd.DataFrame(all_bins, columns=["chrom", "start", "end", "bin_name"])



In [8]:
bins_df

,chrom,start,end,bin_name
0,Chr1,1760,1860,bin_1
1,Chr1,1860,1960,bin_2
2,Chr1,1960,2060,bin_3
3,Chr1,2060,2160,bin_4
4,Chr1,2160,2260,bin_5
...,...,...,...,...
7073895,ChrM,365586,365686,bin_16
7073896,ChrM,365686,365786,bin_17
7073897,ChrM,365786,365886,bin_18
7073898,ChrM,365886,365986,bin_19


In [9]:
# Split the peaks DataFrame based on unique 'tf' values
unique_tfs = peaks['tf'].unique()

# Create a dictionary to store results for each TF
all_results = []

In [12]:
tf_peaks['peak_start'] + tf_peaks['peak_summit'] - 10

0       15645942
1       15645942
2       15643646
3       15643234
4       15880261
          ...   
1994    12843672
1995      546383
1996      545864
1997      546383
1998      544833
Length: 1999, dtype: int64

In [18]:
tf = "AT1G01060"
tf_peaks = peaks[peaks['tf'] == tf]
tf_peak_tmp = pd.DataFrame()
tf_peak_tmp['chr'] = tf_peaks['peak_chr']
tf_peak_tmp['start'] = tf_peaks['peak_start'] + tf_peaks['peak_summit'] - 10
tf_peak_tmp['end'] = tf_peaks['peak_start'] + tf_peaks['peak_summit'] + 10

In [24]:
tf_peak_tmp.to_csv("tf_peak_tmp.bed",index=False, sep="\t", header=False)
tf_peaks_bed = BedTool("tf_peak_tmp.bed")

In [25]:
tf_peaks_bed

<BedTool(tf_peak_tmp.bed)>

In [ ]:
for tf in unique_tfs:
    # Subset the peaks DataFrame for the current TF
    tf_peaks = peaks[peaks['tf'] == tf]
    tf_peaks_tmp = pd.DataFrame(tf_peaks['peak_chr'],tf_peaks['peak_start'] + tf_peaks['peak_summit'] - 10, tf_peaks['peak_start'] + tf_peaks['peak_summit'] + 10)

    # Convert TF-specific peaks to a BEDTool object
    tf_peaks_bed = BedTool(tf_peaks_tmp.to_csv(index=False, sep="\t", header=False))

    # Intersect to count peaks in each bin (using pre-generated bins)
    common_bins_bed = BedTool(common_bins_file)
    overlap = common_bins_bed.intersect(tf_peaks_bed, c=True)

    # Convert the result to a DataFrame
    overlap_df = pd.read_csv(overlap.fn, sep="\t", header=None, names=["chrom", "start", "end", "bin_name", "peak_count"])

    # Summarize peak counts by bin name
    summary_df = overlap_df.groupby("bin_name")["peak_count"].sum().reset_index()

    # Sort bins numerically
    summary_df["bin_number"] = summary_df["bin_name"].str.extract("(\d+)").astype(int)
    summary_df = summary_df.sort_values("bin_number")

    # Add the TF label for the current subset
    summary_df["tf"] = tf

    # Append the result for this TF to the list of results
    all_results.append(summary_df)